In [1]:
import numpy as np
import pandas as pd
#import requests
#from bs4 import BeautifulSoup
from datetime import date, datetime, timedelta
#Con esto podemos echar un vistazo a código HTML directamente en la Notebook
from IPython.display import HTML, display

In [2]:
file = 'final_2016.csv'
data = pd.read_csv(file)
data.head()

,PatientId,AppointmentID,Gender,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,...,Sea Level Pressure low in,Visibility high ml,Visibility avg ml,Visibility low ml,Wind high mph,Wind avg mph,Wind low mph,Precip sum,event,date
0,2.987250e+13,5642903,F,62,JARDIM DA PENHA,0,1,0,0,0,...,29.95,12,7,6,16,11,-,0.01,Nothing,2016-04-29
1,5.589978e+14,5642503,M,56,JARDIM DA PENHA,0,0,0,0,0,...,29.95,12,7,6,16,11,-,0.01,Nothing,2016-04-29
2,4.262962e+12,5642549,F,62,MATA DA PRAIA,0,0,0,0,0,...,29.95,12,7,6,16,11,-,0.01,Nothing,2016-04-29
3,8.679512e+11,5642828,F,8,PONTAL DE CAMBURI,0,0,0,0,0,...,29.95,12,7,6,16,11,-,0.01,Nothing,2016-04-29
4,8.841186e+12,5642494,F,56,JARDIM DA PENHA,0,1,1,0,0,...,29.95,12,7,6,16,11,-,0.01,Nothing,2016-04-29


In [3]:
print(data['DateAppointment'].min(), data['DateAppointment'].max())
print(data['DateScheduled'].min(), data['DateScheduled'].max())
print(data.shape)
print(data.isnull().sum().sum())

2016-04-29 2016-06-08
2015-11-10 2016-06-08
(110527, 38)
0


### Next Steps:
   [1. Reducing number of columns and rebanming some.](#Reducing columns)
   
   [2. Looking for outliers and other inconsistencies.](#some cleaning)
   
   [3. Creating new feature: dayofweek (where necessary).](#DayOfWeek)

   [4. Days in between appointment and schedule (where necessary).](#awaitingTime)

   [5. Mapping (where possible).](#Repeat)
   
   [6. Repeat patinet and previousno-shows.](#mapping)

  [7. More cleaning and maybe Time of day (if possible).](#more)

<a id='Reducing columns'></a>
## 1. Reducing columns.

In [ ]:
### From weather I will only leave 'temp_avg_f', 'wind_avg_mph', 'Precip sum' and 'event' as weather data.

In [4]:
useless_columns = ['dia', 'temp_high_f', 'Temp low F', 'Dew Point high F','Dew_Point_avg_F',
       'Dew Point low F', 'humidity high %','humidity avg %','humidity low %', 'Sea Level Pressure high in',
       'Sea Level Pressure avg (in)', 'Sea Level Pressure low in', 'Visibility avg ml',
       'Visibility high ml', 'Visibility low ml',
       'Wind high mph', 'Wind low mph', 'date']
len(useless_columns)

18

In [5]:
#final_columns = set(data.columns) - set(useless_columns)
final_columns = [c for c in data.columns if c not in useless_columns]
#final_columns = list(final_columns)

print(len(data.columns)-(len(useless_columns)))
print(len(final_columns))

20
20


In [6]:
new_data = data[final_columns]
new_data.head()

,PatientId,AppointmentID,Gender,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,DateScheduled,TimeScheduled,DateAppointment,TimeAppointment,temp_avg_f,Wind avg mph,Precip sum,event
0,2.987250e+13,5642903,F,62,JARDIM DA PENHA,0,1,0,0,0,0,No,2016-04-29,18:38:08,2016-04-29,00:00:00,73,11,0.01,Nothing
1,5.589978e+14,5642503,M,56,JARDIM DA PENHA,0,0,0,0,0,0,No,2016-04-29,16:08:27,2016-04-29,00:00:00,73,11,0.01,Nothing
2,4.262962e+12,5642549,F,62,MATA DA PRAIA,0,0,0,0,0,0,No,2016-04-29,16:19:04,2016-04-29,00:00:00,73,11,0.01,Nothing
3,8.679512e+11,5642828,F,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No,2016-04-29,17:29:31,2016-04-29,00:00:00,73,11,0.01,Nothing
4,8.841186e+12,5642494,F,56,JARDIM DA PENHA,0,1,1,0,0,0,No,2016-04-29,16:07:23,2016-04-29,00:00:00,73,11,0.01,Nothing


In [7]:
len(new_data.columns)

20

In [8]:
 new_data = new_data.rename(columns={'Hipertension': 'Hypertension', 'Neighbourhood': 'Neighborhood', 'Handcap': 'Handicap',\
                        'temp_avg_f': 'Temp_f', 'Wind avg mph': 'wind_mph', 'Precip sum': 'Precip', 'event': 'Event'})

In [9]:
print(new_data['DateAppointment'].min(), new_data['DateAppointment'].max())
print(new_data['DateScheduled'].min(), new_data['DateScheduled'].max())

2016-04-29 2016-06-08
2015-11-10 2016-06-08


In [10]:
new_data.isnull().sum()

PatientId          0
AppointmentID      0
Gender             0
Age                0
Neighborhood       0
Scholarship        0
Hypertension       0
Diabetes           0
Alcoholism         0
Handicap           0
SMS_received       0
No-show            0
DateScheduled      0
TimeScheduled      0
DateAppointment    0
TimeAppointment    0
Temp_f             0
wind_mph           0
Precip             0
Event              0
dtype: int64

<a id='some cleaning'></a>
## 2. Outliers and inconsitencies

In [11]:
##Selecting ####Scholarship is a subsidy given by the goverment and neighborhoods is where the hospital is located
for item in ['Gender', 'Age', 'Neighborhood', 'Scholarship', 'Hypertension', 'Diabetes',\
             'Alcoholism', 'Handicap', 'SMS_received']:
    print(item, len(new_data[item].unique()), new_data[item].unique() )

Gender 2 ['F' 'M']
Age 104 [ 62  56   8  76  23  39  21  19  30  29  22  28  54  15  50  40  46   4
  13  65  45  51  32  12  61  38  79  18  63  64  85  59  55  71  49  78
  31  58  27   6   2  11   7   0   3   1  69  68  60  67  36  10  35  20
  26  34  33  16  42   5  47  17  41  44  37  24  66  77  81  70  53  75
  73  52  74  43  89  57  14   9  48  83  72  25  80  87  88  84  82  90
  94  86  91  98  92  96  93  95  97 102 115 100  99  -1]
Neighborhood 81 ['JARDIM DA PENHA' 'MATA DA PRAIA' 'PONTAL DE CAMBURI' 'REPÚBLICA'
 'GOIABEIRAS' 'ANDORINHAS' 'CONQUISTA' 'NOVA PALESTINA' 'DA PENHA'
 'TABUAZEIRO' 'BENTO FERREIRA' 'SÃO PEDRO' 'SANTA MARTHA' 'SÃO CRISTÓVÃO'
 'MARUÍPE' 'GRANDE VITÓRIA' 'SÃO BENEDITO' 'ILHA DAS CAIEIRAS'
 'SANTO ANDRÉ' 'SOLON BORGES' 'BONFIM' 'JARDIM CAMBURI' 'MARIA ORTIZ'
 'JABOUR' 'ANTÔNIO HONÓRIO' 'RESISTÊNCIA' 'ILHA DE SANTA MARIA'
 'JUCUTUQUARA' 'MONTE BELO' 'MÁRIO CYPRESTE' 'SANTO ANTÔNIO' 'BELA VISTA'
 'PRAIA DO SUÁ' 'SANTA HELENA' 'ITARARÉ' 'INHANGUETÁ' '

In [12]:
# I will fix ages less than 0 and over 100
print(new_data[new_data['Age'] <0].count()['Age'], new_data[new_data['Age'] >100].count()['Age'])
new_data['Age'] = [c if c >=0 else 0 for c in new_data.Age]

1 7


In [13]:
#Obviously, to leave over 100 years is not that common but this outliers I understand are justified.More explanation
new_data[new_data['Age'] > 100]

,PatientId,AppointmentID,Gender,Age,Neighborhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMS_received,No-show,DateScheduled,TimeScheduled,DateAppointment,TimeAppointment,Temp_f,wind_mph,Precip,Event
58014,9.762948e+14,5651757,F,102,CONQUISTA,0,0,0,0,0,0,No,2016-05-03,09:14:53,2016-05-03,00:00:00,70,5,0.0,Nothing
63912,3.196321e+13,5700278,F,115,ANDORINHAS,0,0,0,0,1,0,Yes,2016-05-16,09:17:44,2016-05-19,00:00:00,76,9,0.0,Nothing
63915,3.196321e+13,5700279,F,115,ANDORINHAS,0,0,0,0,1,0,Yes,2016-05-16,09:17:44,2016-05-19,00:00:00,76,9,0.0,Nothing
68127,3.196321e+13,5562812,F,115,ANDORINHAS,0,0,0,0,1,0,Yes,2016-04-08,14:29:17,2016-05-16,00:00:00,80,8,0.0,Nothing
76284,3.196321e+13,5744037,F,115,ANDORINHAS,0,0,0,0,1,0,No,2016-05-30,09:44:51,2016-05-30,00:00:00,77,5,0.0,Nothing
90372,2.342836e+11,5751563,F,102,MARIA ORTIZ,0,0,0,0,0,0,No,2016-05-31,10:19:49,2016-06-02,00:00:00,78,5,0.0,Nothing
97666,7.482346e+14,5717451,F,115,SÃO JOSÉ,0,1,0,0,0,1,No,2016-05-19,07:57:56,2016-06-03,00:00:00,76,6,0.0,Nothing


<a id='DayOfWeek'></a>
## 3. NewFeatures: Day of the Week and month of the year

In [14]:
##Day_of_week, NO big expectations for month_of_the_year because it is mainly one month: May but might be useful in the future
new_data['day_of_week'] = pd.to_datetime(new_data['DateAppointment']).dt.weekday_name
print(new_data['day_of_week'].head())
new_data['month_of_year'] = pd.to_datetime(new_data['DateAppointment']).dt.month
print(new_data['month_of_year'].head())

0    Friday
1    Friday
2    Friday
3    Friday
4    Friday
Name: day_of_week, dtype: object
0    4
1    4
2    4
3    4
4    4
Name: month_of_year, dtype: int64


In [15]:
new_data['day_of_week'] = new_data['day_of_week'].replace({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5,\
                                'Saturday': 6})

<a id='awaitingTime'></a>
## 4. NewFeature: AwaitingTime

In [16]:
new_data['awaiting_time'] =pd.to_datetime(new_data['DateScheduled']) - pd.to_datetime(new_data['DateAppointment'])
new_data['awaiting_time'] = new_data['awaiting_time'].dt.days

In [17]:
new_data['awaiting_time'] = new_data['awaiting_time'] * -1

<a id='mapping'></a>
## 5. Mapping

In [18]:
### target feature
new_data['No-show'] = new_data['No-show'].map({'Yes': 1, 'No': 0})
###Gender Feature
new_data['Gender'] = new_data['Gender'].map({'F': 1, 'M': 0})

In [19]:
new_data['Event'] = new_data['Event'].map({'Nothing': 0, 'Fog': 1, 'Rain': 2, 'Thunderstorm': 3})
new_data['Event'].unique()

array([0, 1, 3, 2])

<a id='Repeat'></a>
## 6. Repeat Patient/Already no-show

In [20]:
new_data['PatientId'] = new_data["PatientId"].astype(int)

new_data['duplicated'] = new_data.duplicated(['PatientId'])##I want all to be marked as duplicate
new_data['duplicated'].head()

new_data['patient_repeat'] = new_data['duplicated'].astype(int)
print(new_data['patient_repeat'].value_counts())

0    62299
1    48228
Name: patient_repeat, dtype: int64


In [21]:
new_data['num_app_missed'] = new_data.groupby('PatientId')['No-show'].apply(lambda x: x.cumsum()-x)

<a id='more'></a>
## 7. More cleaning: Dropping unnecessary columns and dumming it up

In [22]:
#I am going to do it in case th time for scheduled appointment would correspond to the actual time of appointment:
#0 Night, 1 Morning 2 Afternoon and 3 evening
new_data['TimeScheduled'] = pd.to_datetime(new_data['TimeScheduled'])
new_data = new_data.assign(session=pd.cut(new_data.TimeScheduled.dt.hour,\
                               [0,6,12,18,24],labels=[0,1,2,3]))

In [23]:
##It is telling us that most of the apointments were scheduled during morning and afternoon what has a lot of sense
new_data['session'].value_counts()

1    72325
2    36033
0     1578
3      591
Name: session, dtype: int64

In [24]:
new_data.shape

(110527, 27)

In [25]:
new_data['session'] = new_data['session'].astype(float)

In [26]:
new_data[['TimeScheduled', 'session']].head()

,TimeScheduled,session
0,2017-12-16 18:38:08,2.0
1,2017-12-16 16:08:27,2.0
2,2017-12-16 16:19:04,2.0
3,2017-12-16 17:29:31,2.0
4,2017-12-16 16:07:23,2.0


In [27]:
new_data.isnull().sum().sum()

0

# Dropping unnecessary columns: duplicated and Time Appointment.
        ## The first one is replace by Patient Repeat
        ## The second one has no use if is only 00.

In [28]:
new_data.drop(['duplicated', 'TimeAppointment'], axis=1, inplace=True)

In [ ]:
print(new_data.shape)

In [30]:
new_data.dtypes

PatientId                   int64
AppointmentID               int64
Gender                      int64
Age                         int64
Neighborhood               object
Scholarship                 int64
Hypertension                int64
Diabetes                    int64
Alcoholism                  int64
Handicap                    int64
SMS_received                int64
No-show                     int64
DateScheduled              object
TimeScheduled      datetime64[ns]
DateAppointment            object
Temp_f                      int64
wind_mph                    int64
Precip                    float64
Event                       int64
day_of_week                 int64
month_of_year               int64
awaiting_time               int64
patient_repeat              int64
num_app_missed              int64
session                   float64
dtype: object

## Separating columns by type

# Dummies: day of week, neighborhood, ...

In [31]:
rars = new_data.select_dtypes(exclude=['object', 'float', 'integer'])
print(rars.columns, len(rars.columns))
objectes = new_data.select_dtypes(include=['object'])
print(objectes.columns, len(objectes.columns))
numeric = new_data.select_dtypes(include=['float', 'integer'])
print(numeric.columns, len(numeric.columns))

Index(['TimeScheduled'], dtype='object') 1
Index(['Neighborhood', 'DateScheduled', 'DateAppointment'], dtype='object') 3
Index(['PatientId', 'AppointmentID', 'Gender', 'Age', 'Scholarship',
       'Hypertension', 'Diabetes', 'Alcoholism', 'Handicap', 'SMS_received',
       'No-show', 'Temp_f', 'wind_mph', 'Precip', 'Event', 'day_of_week',
       'month_of_year', 'awaiting_time', 'patient_repeat', 'num_app_missed',
       'session'],
      dtype='object') 21


In [32]:
renew_data = pd.concat([new_data, pd.get_dummies(new_data['day_of_week'], prefix='dow'),\
                  pd.get_dummies(new_data['Neighborhood'], prefix='hood'), pd.get_dummies(new_data['Event'], prefix='event')], axis=1)
renew_data.head()

,PatientId,AppointmentID,Gender,Age,Neighborhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,...,hood_SÃO CRISTÓVÃO,hood_SÃO JOSÉ,hood_SÃO PEDRO,hood_TABUAZEIRO,hood_UNIVERSITÁRIO,hood_VILA RUBIM,event_0,event_1,event_2,event_3
0,29872499824296,5642903,1,62,JARDIM DA PENHA,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,558997776694438,5642503,0,56,JARDIM DA PENHA,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,4262962299951,5642549,1,62,MATA DA PRAIA,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,867951213174,5642828,1,8,PONTAL DE CAMBURI,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,8841186448183,5642494,1,56,JARDIM DA PENHA,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [33]:
renew_data.isnull().sum().sum()

0

In [34]:
renew_data.to_csv('appointments2016.csv',index=False, encoding='utf-8')